In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta, date

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import sys
sys.path.append('../')

from axia.shifted_beta_geometric import ShiftedBetaGeometric
from axia.sbg_survival import SBGSurvival
from axia import util
from axia import report
from axia import loader

from importlib import reload

# Data: Load, Preprocess, and Split

In [2]:
jobber_data = loader.JobberDataLoader(path="../data/")
raw_data = jobber_data.get_data()
raw_data = raw_data.sample(100, random_state=189723)

In [76]:
%%time
reload(util)
data = util.SubscriptionData(
    data=raw_data,
    start_date_col="first_paying_date",
    end_date_col="last_churn_date",
    subscription_initial="initial_subscription_amt",
    subscription_current="current_subscription_amt",
    additional_cols=[raw_data["frequency"].astype("category")] + [
        raw_data[col].apply(func).astype("category")
        for col, func in jobber_data.transformations.items()
    ],
    split_at=datetime(2018, 1, 1),
#     split_at=datetime.now(),
)

CPU times: user 1.46 s, sys: 4.16 ms, total: 1.47 s
Wall time: 1.47 s


In [65]:
raw_data.loc[1802]

country                                Australia
industry                             Renovations
account_created_date         2013-01-30 00:00:00
first_paying_date            2013-03-02 00:00:00
last_churn_date              2016-05-20 00:00:00
days_to_paying                                31
initial_subscription_amt                      38
current_subscription_amt                      29
frequency                                monthly
current_billing_frequency                monthly
is_paying                                      0
paying_months                                 29
tot_revenue                                 1003
Name: 1802, dtype: object

In [50]:
data.cdtr.sort_values("end_date", ascending=False)

start_date   end_date  age  alive  is_starting_month  \
account_id end_of_month                                                        
42851      2017-04-30   2015-12-01 2017-12-01   16      1                  0   
           2016-02-29   2015-12-01 2017-12-01    2      1                  0   
           2016-04-30   2015-12-01 2017-12-01    4      1                  0   
           2016-05-31   2015-12-01 2017-12-01    5      1                  0   
           2016-07-31   2015-12-01 2017-12-01    7      1                  0   
           2016-08-31   2015-12-01 2017-12-01    8      1                  0   
           2016-09-30   2015-12-01 2017-12-01    9      1                  0   
           2016-10-31   2015-12-01 2017-12-01   10      1                  0   
           2016-11-30   2015-12-01 2017-12-01   11      1                  0   
           2016-12-31   2015-12-01 2017-12-01   12      1                  0   
           2017-01-31   2015-12-01 2017-12-01   13      1                  0   
           2017-02-28   2015-12-01 2017-12-01   14      1                  0   
           2017-03-31   2015-12-01 2017-12-01   15      1                  0   
           2017-05-31   2015-12-01 2017-12-01   17      1                  0   
           2017-06-30   2015-12-01 2017-12-01   18      1                  0   
           2017-07-31   2015-12-01 2017-12-01   19      1                  0   
           2017-08-31   2015-12-01 2017-12-01   20      1                  0   
           2017-09-30   2015-12-01 2017-12-01   21      1                  0   
           2017-10-31   2015-12-01 2017-12-01   22      1                  0   
           2017-11-30   2015-12-01 2017-12-01   23      1                  0   
           2017-12-31   2015-12-01 2017-12-01   24      0                  0   
52643      2016-09-30   2016-06-01 2017-12-01    3      1                  0   
           2016-08-31   2016-06-01 2017-12-01    2      1                  0   
           2016-07-31   2016-06-01 2017-12-01    1      1                  0   
           2016-06-30   2016-06-01 2017-12-01    0      1                  1   
42851      2016-03-31   2015-12-01 2017-12-01    3      1                  0   
           2016-06-30   2015-12-01 2017-12-01    6      1                  0   
           2016-01-31   2015-12-01 2017-12-01    1      1                  0   
52643      2017-05-31   2016-06-01 2017-12-01   11      1                  0   
           2017-12-31   2016-06-01 2017-12-01   18      0                  0   
...                            ...        ...  ...    ...                ...   
54792      2016-10-31   2016-07-01        NaT    3      1                  0   
           2016-11-30   2016-07-01        NaT    4      1                  0   
           2016-12-31   2016-07-01        NaT    5      1                  0   
           2017-01-31   2016-07-01        NaT    6      1                  0   
           2017-02-28   2016-07-01        NaT    7      1                  0   
           2017-03-31   2016-07-01        NaT    8      1                  0   
           2017-04-30   2016-07-01        NaT    9      1                  0   
           2017-05-31   2016-07-01        NaT   10      1                  0   
           2017-06-30   2016-07-01        NaT   11      1                  0   
           2017-07-31   2016-07-01        NaT   12      1                  0   
           2017-08-31   2016-07-01        NaT   13      1                  0   
           2017-09-30   2016-07-01        NaT   14      1                  0   
           2017-10-31   2016-07-01        NaT   15      1                  0   
           2017-11-30   2016-07-01        NaT   16      1                  0   
           2017-12-31   2016-07-01        NaT   17      1                  0   
62851      2016-10-31   2016-10-01        NaT    0      1                  1   
           2016-11-30   2016-10-01        NaT    1      1                  0   
           2016-12-31   

In [41]:
data.cdtr.max()

start_date                   2017-12-01 00:00:00
end_date                     2019-02-01 00:00:00
age                                           64
alive                                          1
is_starting_month                              1
is_cancelation_month                           1
months_alive                                  65
subscription_value_base                      210
subscription_value_linear                    210
value_to_date_base                          9450
value_to_date_linear                     8528.28
dtype: object

In [48]:
data.dva.max()

start_date             2019-02-01
end_date               2019-02-01
age                           NaT
alive                         NaT
subscription_initial          NaT
subscription_last             NaT
subscription_current          NaT
dtype: datetime64[ns]

In [21]:
df = data.dtr.copy()

In [31]:
df['end_date'] = df["end_date"].apply(lambda dt: dt if dt < datetime(2018, 1, 1) else None)

In [34]:
df.head()

,start_date,end_date,age,alive,subscription_initial,subscription_last,subscription_current,frequency,country,industry
account_id,,,,,,,,,,
14934,2017-04-01,NaT,23,1,96.0,96.0,96.0,monthly,united_states,other_industry
6233,2014-02-01,NaT,61,1,44.0,53.0,53.0,monthly,united_states,pressure_washing_service
44628,2016-01-01,2016-04-01,3,0,59.0,59.0,0.0,monthly,united_states,other
42851,2015-12-01,2017-12-01,24,0,48.0,48.0,0.0,annual,united_states,other
86400,2017-07-01,NaT,13,0,58.0,58.0,0.0,annual,united_states,residential_cleaning


In [33]:
data.dtr

,start_date,end_date,age,alive,subscription_initial,subscription_last,subscription_current,frequency,country,industry
account_id,,,,,,,,,,
14934,2017-04-01,NaT,23,1,96.0,96.0,96.0,monthly,united_states,other_industry
6233,2014-02-01,NaT,61,1,44.0,53.0,53.0,monthly,united_states,pressure_washing_service
44628,2016-01-01,2016-04-01,3,0,59.0,59.0,0.0,monthly,united_states,other
42851,2015-12-01,2017-12-01,24,0,48.0,48.0,0.0,annual,united_states,other
86400,2017-07-01,2018-08-01,13,0,58.0,58.0,0.0,annual,united_states,residential_cleaning
35508,2015-07-01,NaT,44,1,39.0,48.0,48.0,annual,united_states,other_industry
77264,2017-04-01,NaT,23,1,69.0,78.0,78.0,monthly,united_states,window_washing
63231,2016-10-01,NaT,29,1,73.0,91.0,91.0,monthly,united_kingdom,construction_&_contracting
84665,2017-07-01,NaT,20,1,123.0,123.0,123.0,monthly,canada,landscaping_contractor
